In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 상담 후처리 지원 - 상담 결과 개선점 파악 및 관련 내용 보고 시스템

In [ ]:
import json

conversation_path = '/content/drive/MyDrive/데이터/ch0703/conversation.json'
with open(conversation_path, "r") as file:
  data = json.load(file)

In [ ]:
print(data[0])

{'id': 1, 'name': 'John Doe', 'content': [{'role': 'user', 'content': '안녕하세요. 오늘 날짜로 인천에서 출발하는 뉴욕행 비행기를 예약할 수 있나요?'}, {'role': 'assistant', 'content': '안녕하세요! 오늘 날짜로 인천에서 뉴욕으로 가는 비행기는 다음과 같은 옵션이 있습니다:\n\n1. 대한항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착\n   - 출발 시간: 오후 1:30, 도착 시간: 오후 3:30\n   - 출발 시간: 오후 5:00, 도착 시간: 오후 7:00\n   - 출발 시간: 오후 9:00, 도착 시간: 오후 11:00\n   - 출발 시간: 오전 12:00 (다음날), 도착 시간: 오전 2:00 (다음날)\n   - 출발 시간: 오전 3:00 (다음날), 도착 시간: 오전 5:00 (다음날)\n   - 출발 시간: 오전 6:00 (다음날), 도착 시간: 오전 8:00 (다음날)\n\n2. 유나이티드 항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착 \n   - 출발 시간: 오후 7:00, 도착 시간: 오후 9:00\n   - 출발 시간: 오전 2:00 (다음날), 도착 시간: 오전 4:00 (다음날)\n   - 출발 시간: 오전 8:00 (다음날), 도착 시간: 오전 10:00 (다음날)\n\n위 옵션 중 원하시는 시간대를 선택하셔서 예약하시면 됩니다. 추가로 필요로 하는 정보가 있으시다면 언제든지 말씀해주세요!'}, {'role': 'user', 'content': '대한항공으로 예약해 주세요'}, {'role': 'assistant', 'content': '대한항공을 이용하시길 원하신다면, 여러 시간대의 옵션이 있어 선택하실 수 있습니다. 구체적으로 어느 항공편의 시간대로 예약을 도와드릴까요? 예를 들어, 오후 1:30 혹은 오후 5:00 등 원하시는 시간을 알려주시면 상세 예약 절차를 안내해 드리겠습니다.'}, {'role': 'user', 

##### 1.  고객 불만 정도 감지

In [ ]:
from string import Template

prompt = Template("""
너는 사용자와 인공지능 상담 봇 간의 대화를 분석해서 사용자의 만족도를 측정하는 인공지능이야.
아래 대화 내용을 분석해서 사용자의 만족도를 1부터 5까지 숫자로 측정해줘.
5: 챗봇의 답변이 매우 도움이 되었고 사용자의 반응도 긍정적이였다
4: 챗봇의 답변이 상담에 도움이 되었다
3: 챗봇이 사용자의 의도에 맞는 답변을 하였다
2: 챗봇의 답변이 사용자의 질문 의도와 맞지 않는 부분이있다.
1: 챗봇의 답변이 사용자의 질문과 맞지 않아 사용자가 불만스러운 반응을 보였다

{
  점수: ...
}

대화 내용: $conv_hist
""")


In [ ]:
def change_format(conv_hist):
  conv_hist_str = ""
  for i in range(len(conv_hist)):
    if conv_hist[i]['role'] == "user":
        conv_hist_str += "사용자 : " + conv_hist[i]['content'] + "\n"
    else:
        conv_hist_str += "인공지능 : " + conv_hist[i]['content'] + "\n"

    if i % 2 == 1:
      conv_hist_str += "\n\n"
  return conv_hist_str

print(change_format(data[0]['content']))

사용자 : 안녕하세요. 오늘 날짜로 인천에서 출발하는 뉴욕행 비행기를 예약할 수 있나요?
인공지능 : 안녕하세요! 오늘 날짜로 인천에서 뉴욕으로 가는 비행기는 다음과 같은 옵션이 있습니다:

1. 대한항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착
   - 출발 시간: 오후 1:30, 도착 시간: 오후 3:30
   - 출발 시간: 오후 5:00, 도착 시간: 오후 7:00
   - 출발 시간: 오후 9:00, 도착 시간: 오후 11:00
   - 출발 시간: 오전 12:00 (다음날), 도착 시간: 오전 2:00 (다음날)
   - 출발 시간: 오전 3:00 (다음날), 도착 시간: 오전 5:00 (다음날)
   - 출발 시간: 오전 6:00 (다음날), 도착 시간: 오전 8:00 (다음날)

2. 유나이티드 항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착 
   - 출발 시간: 오후 7:00, 도착 시간: 오후 9:00
   - 출발 시간: 오전 2:00 (다음날), 도착 시간: 오전 4:00 (다음날)
   - 출발 시간: 오전 8:00 (다음날), 도착 시간: 오전 10:00 (다음날)

위 옵션 중 원하시는 시간대를 선택하셔서 예약하시면 됩니다. 추가로 필요로 하는 정보가 있으시다면 언제든지 말씀해주세요!


사용자 : 대한항공으로 예약해 주세요
인공지능 : 대한항공을 이용하시길 원하신다면, 여러 시간대의 옵션이 있어 선택하실 수 있습니다. 구체적으로 어느 항공편의 시간대로 예약을 도와드릴까요? 예를 들어, 오후 1:30 혹은 오후 5:00 등 원하시는 시간을 알려주시면 상세 예약 절차를 안내해 드리겠습니다.


사용자 : 가장 빠른 시간대로 예약해 주세요
인공지능 : 가장 빠른 시간대인 대한항공의 오후 1:30 비행기(INC → JFK)로 예약을 진행하겠습니다. 이 비행은 오늘 인천국제공항에서 출발해서 존 F. 케네디 국제공항에 오후 3:30에 도착합니다.

저는 실제로 예약을 처리할 수 없으므로, 예약을 완료하시려면 대한항공의 공

In [ ]:
from openai import OpenAI

OPENAI_API_KEY= # OpenAI API Key
model_name="gpt-3.5-turbo"

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def measure_satisfaction(conv_hist, prompt):
    chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
          {
              "role": "user",
              "content": prompt.substitute(conv_hist = change_format(conv_hist))
          }
      ],
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
    )
    return chat_completion.choices[0].message.content

print(measure_satisfaction(data[0]['content'], prompt))

{
  "score": 4
}


##### 2. 사용자의 질의 주제 및 요청 의도를 요약하기

In [ ]:
from string import Template

prompt_for_summary = Template("""
주어진 사용자의 요청에 대한 키워드와 요약을 아래와 같은 형태로 정리해줘

주제는 아래의 네 가지 종류 중 하나에서 골라서 기록해줘

- 항공편 예약
- 항공편 취소
- 항공편 변경
- 기타

{
  주제: ...
  요청 사항: ...
}

입력: $input
""")

print(prompt_for_summary.substitute(input=data[0]['content']))


주어진 사용자의 요청에 대한 키워드와 요약을 아래와 같은 형태로 정리해줘

주제는 아래의 네 가지 종류 중 하나에서 골라서 기록해줘

- 항공편 예약
- 항공편 취소
- 항공편 변경
- 기타

{
  주제: ...
  요청 사항: ...
}

입력: [{'role': 'user', 'content': '안녕하세요. 오늘 날짜로 인천에서 출발하는 뉴욕행 비행기를 예약할 수 있나요?'}, {'role': 'assistant', 'content': '안녕하세요! 오늘 날짜로 인천에서 뉴욕으로 가는 비행기는 다음과 같은 옵션이 있습니다:\n\n1. 대한항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착\n   - 출발 시간: 오후 1:30, 도착 시간: 오후 3:30\n   - 출발 시간: 오후 5:00, 도착 시간: 오후 7:00\n   - 출발 시간: 오후 9:00, 도착 시간: 오후 11:00\n   - 출발 시간: 오전 12:00 (다음날), 도착 시간: 오전 2:00 (다음날)\n   - 출발 시간: 오전 3:00 (다음날), 도착 시간: 오전 5:00 (다음날)\n   - 출발 시간: 오전 6:00 (다음날), 도착 시간: 오전 8:00 (다음날)\n\n2. 유나이티드 항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착 \n   - 출발 시간: 오후 7:00, 도착 시간: 오후 9:00\n   - 출발 시간: 오전 2:00 (다음날), 도착 시간: 오전 4:00 (다음날)\n   - 출발 시간: 오전 8:00 (다음날), 도착 시간: 오전 10:00 (다음날)\n\n위 옵션 중 원하시는 시간대를 선택하셔서 예약하시면 됩니다. 추가로 필요로 하는 정보가 있으시다면 언제든지 말씀해주세요!'}, {'role': 'user', 'content': '대한항공으로 예약해 주세요'}, {'role': 'assistant', 'content': '대한항공을 이용하시길 원하신다면, 여러 시간대의 옵션이 있어 선택하실 수 있습니다. 구체적으로 어느 항공편의 시간

In [ ]:
def classify_user_intent(conv_hist):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": prompt_for_summary.substitute(input=conv_hist)}
    ]
  )

  return response.choices[0].message.content

print(classify_user_intent(data[0]['content']))

{
  "주제": "항공편 예약",
  "요청 사항": "인천에서 출발하는 뉴욕행 비행기를 예약해 주세요."
}


##### 3. 상담 후 처리 하기

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ConvHistDataset(Dataset):
    def __init__(self, conv_hist_list):
        self.conv_hist_list = conv_hist_list

    def __len__(self):
        return len(self.conv_hist_list)

    def __getitem__(self, idx):
        return self.conv_hist_list[idx]

convHistDataset = ConvHistDataset(data)

In [ ]:
for conv in convHistDataset:
  print(conv)
  break


{'id': 1, 'name': 'John Doe', 'content': [{'role': 'user', 'content': '안녕하세요. 오늘 날짜로 인천에서 출발하는 뉴욕행 비행기를 예약할 수 있나요?'}, {'role': 'assistant', 'content': '안녕하세요! 오늘 날짜로 인천에서 뉴욕으로 가는 비행기는 다음과 같은 옵션이 있습니다:\n\n1. 대한항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착\n   - 출발 시간: 오후 1:30, 도착 시간: 오후 3:30\n   - 출발 시간: 오후 5:00, 도착 시간: 오후 7:00\n   - 출발 시간: 오후 9:00, 도착 시간: 오후 11:00\n   - 출발 시간: 오전 12:00 (다음날), 도착 시간: 오전 2:00 (다음날)\n   - 출발 시간: 오전 3:00 (다음날), 도착 시간: 오전 5:00 (다음날)\n   - 출발 시간: 오전 6:00 (다음날), 도착 시간: 오전 8:00 (다음날)\n\n2. 유나이티드 항공, 인천국제공항 출발, 존 F. 케네디 국제공항 도착 \n   - 출발 시간: 오후 7:00, 도착 시간: 오후 9:00\n   - 출발 시간: 오전 2:00 (다음날), 도착 시간: 오전 4:00 (다음날)\n   - 출발 시간: 오전 8:00 (다음날), 도착 시간: 오전 10:00 (다음날)\n\n위 옵션 중 원하시는 시간대를 선택하셔서 예약하시면 됩니다. 추가로 필요로 하는 정보가 있으시다면 언제든지 말씀해주세요!'}, {'role': 'user', 'content': '대한항공으로 예약해 주세요'}, {'role': 'assistant', 'content': '대한항공을 이용하시길 원하신다면, 여러 시간대의 옵션이 있어 선택하실 수 있습니다. 구체적으로 어느 항공편의 시간대로 예약을 도와드릴까요? 예를 들어, 오후 1:30 혹은 오후 5:00 등 원하시는 시간을 알려주시면 상세 예약 절차를 안내해 드리겠습니다.'}, {'role': 'user', 

In [ ]:
class ConvSummaryBot:
  def __init__(self):
    self.stat_topic = {
        "항공편 예약" : 0,
        "항공편 취소" : 0,
        "항공편 변경" : 0,
        "기타" : 0,
    }

    self.stat_satisfaction = {
        "5" : 0,
        "4" : 0,
        "3" : 0,
        "2" : 0,
        "1" : 0,
    }

    self.prompt =  Template("""
      너는 사용자와 인공지능 상담 봇 간의 대화를 분석해서 사용자의 만족도를 측정하는 인공지능이야.
      아래 대화 내용을 분석해서 사용자의 만족도를 1부터 5까지 숫자로 측정해줘.
      5: 챗봇의 답변이 매우 도움이 되었고 사용자의 반응도 긍정적이였다
      4: 챗봇의 답변이 상담에 도움이 되었다
      3: 챗봇이 사용자의 의도에 맞는 답변을 하였다
      2: 챗봇의 답변이 사용자의 질문 의도와 맞지 않는 부분이있다.
      1: 챗봇의 답변이 사용자의 질문과 맞지 않아 사용자가 불만스러운 반응을 보였다

      {
        점수: ...
      }

      대화 내용: $conv_hist
      """)

    self.prompt_for_summary = Template("""
      주어진 사용자의 요청에 대한 키워드와 요약을 아래와 같은 형태로 정리해줘

      주제는 아래의 네 가지 종류 중 하나에서 골라서 기록해줘

      - 항공편 예약
      - 항공편 취소
      - 항공편 변경
      - 기타

      {
        주제: ...
        요청 사항: ...
      }

      입력: $input
      """)

  def change_format(self, conv_hist):
    conv_hist_str = ""
    for i in range(len(conv_hist)):
      if conv_hist[i]['role'] == "user":
          conv_hist_str += "사용자 : " + conv_hist[i]['content'] + "\n"
      else:
          conv_hist_str += "인공지능 : " + conv_hist[i]['content'] + "\n"

      if i % 2 == 1:
        conv_hist_str += "\n\n"
    return conv_hist_str

  def measure_satisfaction(self, conv_hist):
    chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
          {
              "role": "user",
              "content": self.prompt.substitute(conv_hist = change_format(conv_hist))
          }
      ],
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
    )

    response_json = json.loads(chat_completion.choices[0].message.content)
    try:
      point = response_json["점수"]
    except:
      point = response_json["score"]

    self.stat_satisfaction[str(point)] += 1
    return chat_completion.choices[0].message.content

  def classify_user_intent(self, conv_hist):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": self.prompt_for_summary.substitute(input=conv_hist)}
      ]
    )

    response_json = json.loads(response.choices[0].message.content)

    try:
      self.stat_topic[response_json["주제"]] += 1
    except:
      self.stat_topic["기타"] += 1

convSummaryBot = ConvSummaryBot()

In [ ]:
for conv in convHistDataset:
  convSummaryBot.measure_satisfaction(conv['content'])
  convSummaryBot.classify_user_intent(conv['content'])

In [ ]:
convSummaryBot.stat_topic

{'항공편 예약': 15, '항공편 취소': 10, '항공편 변경': 0, '기타': 1}

In [ ]:
convSummaryBot.stat_satisfaction

{'5': 0, '4': 11, '3': 15, '2': 0, '1': 0}